<a href="https://colab.research.google.com/github/abattiasia/KI-Kurs-Mystro/blob/main/Mobile_Price_Range_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'mobile-price-range:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5461486%2F9057325%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240917%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240917T091915Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D00c795c2a98d554c40dd97346bb5af69b275d7513446c4ae6b9ce8754f96273b0a1ceb6ceb770a3565b1beae2e5486fa4719253fdca1b2a588907dbbb7bdc2e2eb64b92617424e654a1291e84fc173766322dfc3295fc63636936789fd8cb0e3a6be3ece01cb384eaece2386e30f4c708535db8bfd2eafa45a0d34819d312f9993dd20c3859e9b0d7d64e06f2007dde035b3367ced26a54bd700f91aeccfb006b16d870933d545ff9a281a8dbf5e423e5de0136b912932490d3177e21f318d9a2a21984eb3f8e4d9c620cb801b3f6e5ce0446eb92d4a75e52ebe456e5e390e596dedaa1d4b03e7e350061099fdd5fcc04c4d5b432296059d486d2c0f4cdc9353'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier

# Mobile Price Range

About Dataset
The .csv file contains data on 2000 mobile and their specifications. The price of a phone become a necessity in our daily lives and the price of that depends on different factors which need to be consider.

In [ ]:
# read data
dftrain = pd.read_csv('/kaggle/input/mobile-price-range/train.csv')
dftrain.head()

In [ ]:
dftrain.describe

In [ ]:
# read data
dftest = pd.read_csv('/kaggle/input/mobile-price-range/test.csv')
dftest.head()

In [ ]:
dftest.describe

In [ ]:
dftrain.columns

In [ ]:
dftest.columns

In [ ]:
Xtr=dftrain.drop(columns=['price_range'])
ytr=dftrain['price_range']

In [ ]:
# Xte=dftest.drop(columns=['id'])
# #yte=dftest['price_range']

In [ ]:
x=dftrain.drop(columns=['price_range'])
y=dftrain['price_range']

X_train , X_test , y_train , y_test = train_test_split(x,y,test_size=.20)

# 01 KNN Prject  K Neighbors Classifier  -> model=KNeighborsClassifier()

In [ ]:
# build model
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train , y_train)

In [ ]:
# predict
y_pred = model.predict(X_test)
y_pred

In [ ]:
# accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test , y_pred)

In [ ]:
## choose best k

# Define the range of k values to test
k_values = range(1, 10)
test_accuracies = []

# Train the k-NN classifier with different values of k and evaluate on the test set
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)  # Train the model
    y_pred = knn.predict(X_test)  # Predict on the test set
    accuracy = accuracy_score(y_test, y_pred)  # Calculate accuracy
    test_accuracies.append(accuracy)

# Determine the best k
best_k = k_values[np.argmax(test_accuracies)]
print(f"The best k value is: {best_k}")

In [ ]:
# Plot the k values vs. accuracy on the test set
plt.figure(figsize=(10, 6))
plt.plot(k_values, test_accuracies, marker='o')
plt.xlabel('k')
plt.ylabel('Test Set Accuracy')
plt.title('k-NN Varying k Value')
plt.show()

# 02 Tree Prject  Decision  Tree  Classifier -> model=DecisionTreeClassifier()

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier


# model
model = DecisionTreeClassifier()
model.fit(X_train,y_train)

In [ ]:
# predict and accuracy
y_pred = model.predict(X_test)
y_pred

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
# plot tree
from sklearn import tree

plt.figure(figsize=(20, 10))
tree.plot_tree(model)
plt.show()

# 03 Randum Project Random Forest --> mdel=RandomForestClassifier()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(criterion='entropy',n_estimators=30)
model.fit(X_train,y_train)

In [ ]:
# predict and accuracy
y_pred = model.predict(X_test)
y_pred

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
# Get feature importances
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

# Print feature importance
print("Feature importances:")
for i in indices:
    print(f"{x.columns[i]}: {importances[i]}")

In [ ]:
# Plot feature importances
plt.figure(figsize=(10, 6))
plt.title("Feature Importances")
plt.bar(range(x.shape[1]), importances[indices], align="center")
plt.xticks(range(x.shape[1]), np.array(x.columns)[indices], rotation=45)
plt.xlim([-1, x.shape[1]])
plt.show()

# Comparison  between the 3 Models

KNeighborsClassifier() accuracy =  92%

DecisionTreeClassifier() accuracy = 81.5%

RandomForestClassifier()  accuracy = 82.57%